<a href="https://colab.research.google.com/github/Tobi-KL/SOC_Profile_Generation/blob/main/Run_SOC_Profile_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import packages and change directory

In [2]:
import sys
import os

# Input path directory
path_directory = "/content/drive/MyDrive/Tobias_MA/Abgabe/SOC_Profile_Generation"

sys.path.insert(0, path_directory)
os.chdir(path_directory)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import csv
import os.path

from classes.household import Household
from functions.rank_households import rank_households
from functions.rank_households_all import rank_households_all
from functions.create_soc_profiles import create_soc_profiles
#from functions.aggregated_profiles import aggregated_profiles
from functions.aggregated_profiles_day import aggregated_profiles_day
from functions.aggregated_profiles_week import aggregated_profiles_week
from functions.aggregated_profiles_strategies import aggregated_profiles_strategies
from functions.aggregated_profiles_lvp import aggregated_profiles_lvp

### Load data

In [4]:
### Mobility Data

mobility_data_pkl_path =  path_directory + '/inputs/data_mop_priority.pkl'


### TANK Data (MOP):
# needed for car segment (in column [174])
mob_car_info_csv_path = path_directory + '/inputs/TANK18.csv'

### Electric Car database for car substitution
# [:,3]: Battery capacity [kWh]
# [:,4]: WLTP-consumption [kWh / 100km]
# [:,5]: Car Charging power [kW]
el_cars_csv_path = path_directory + '/inputs/Elektroauto_Datenbank.csv'

### Temperature [°C] in time period (same timesteps)
weather_csv_path = path_directory + '/inputs/Temperaturen_Deutschland_2017.csv'

### Timesteps
# Input timestep length and number of timesteps
ts_length = 10
no_of_ts = 1008
#no_of_ts = int (states_mop.shape[1])   # needs to process data first

In [5]:
### Data Processing

# pkl file
with open(mobility_data_pkl_path, 'rb') as input:
        [states_mop,speed_mop,meta_mop,meta_header_mop] = pickle.load(input)
# observe only year 2017
meta_mop_data = meta_mop[-3820:-1,]
states_mop_data = states_mop[-3820:-1,]
speed_mop_data = speed_mop[-3820:-1,]

# csv files
mob_car_info_csv = np.genfromtxt (mob_car_info_csv_path, 
                                  delimiter=";", 
                                  encoding = "ISO-8859-1")

el_cars_csv = np.genfromtxt (el_cars_csv_path, 
                             delimiter=";", 
                             encoding = "ISO-8859-1")

weather_csv = np.genfromtxt (weather_csv_path, 
                             delimiter=";", 
                             encoding = "ISO-8859-1")

# sort data by weekdays
for i in range(len(states_mop_data)):
    dates = meta_mop_data[i,32:39]
    dates = pd.to_timedelta(dates, unit='D') + pd.Timestamp('1960-1-1')
    weekdays = dates.strftime("%A")
    if (weekdays[0] == "Sunday"):
        part_1 = states_mop_data[i,144:]
        part_2 = states_mop_data[i,0:144]
        states_mop_data[i] = np.concatenate((part_1, part_2))
        part_1_speeds = speed_mop_data[i,144:]
        part_2_speeds = speed_mop_data[i,0:144]
        speed_mop_data[i] = np.concatenate((part_1_speeds, part_2_speeds)) 
    elif (weekdays[0] == "Saturday"):
        part_1 = states_mop_data[i,288:]
        part_2 = states_mop_data[i,0:288]
        states_mop_data[i] = np.concatenate((part_1, part_2))
        part_1_speeds = speed_mop_data[i,288:]
        part_2_speeds = speed_mop_data[i,0:288]
        speed_mop_data[i] = np.concatenate((part_1_speeds, part_2_speeds)) 
    elif (weekdays[0] == "Friday"):
        part_1 = states_mop_data[i,432:]
        part_2 = states_mop_data[i,0:432]
        states_mop_data[i] = np.concatenate((part_1, part_2))
        part_1_speeds = speed_mop_data[i,432:]
        part_2_speeds = speed_mop_data[i,0:432]
        speed_mop_data[i] = np.concatenate((part_1_speeds, part_2_speeds)) 
    elif (weekdays[0] == "Thursday"):
        part_1 = states_mop_data[i,576:]
        part_2 = states_mop_data[i,0:576]
        states_mop_data[i] = np.concatenate((part_1, part_2))
        part_1_speeds = speed_mop_data[i,576:]
        part_2_speeds = speed_mop_data[i,0:576]
        speed_mop_data[i] = np.concatenate((part_1_speeds, part_2_speeds)) 
    elif (weekdays[0] == "Wednesday"):
        part_1 = states_mop_data[i,720:]
        part_2 = states_mop_data[i,0:720]
        states_mop_data[i] = np.concatenate((part_1, part_2))
        part_1_speeds = speed_mop_data[i,720:]
        part_2_speeds = speed_mop_data[i,0:720]
        speed_mop_data[i] = np.concatenate((part_1_speeds, part_2_speeds)) 
    elif (weekdays[0] == "Tuesday"):
        part_1 = states_mop_data[i,864:]
        part_2 = states_mop_data[i,0:864]
        states_mop_data[i] = np.concatenate((part_1, part_2))
        part_1_speeds = speed_mop_data[i,864:]
        part_2_speeds = speed_mop_data[i,0:864]
        speed_mop_data[i] = np.concatenate((part_1_speeds, part_2_speeds))

### Rank households

In [ ]:
# Rank households according to your input

# Choose the following main parameters (will be fulfilled)
input_number_of_occupants = 1
input_number_of_drivers = 1
input_number_of_cars = 1


# Choose the following five soft parameters and their weight.
# Fitting households will be sorted by weighted soft parameters.
# Sum of all weights = 1.0
# Find codes in Codeplan_SOC_Profile_Generation.md on github
input_income = 1
input_w_income = 0.2

input_population = 9
input_w_population = 0.2

input_year_of_birth = 1980
input_w_year_of_birth = 0.2

input_job = 1
input_w_job = 0.2

# [km / week]
input_distance = 1000
input_w_distance = 0.2

# How many households should be returned?
# Put "all", if all fitting households should be returned
quantity = 100

###

households = rank_households(meta_mop_data,
                             states_mop_data,
                             speed_mop_data,
                             no_of_ts, 
                             ts_length,
                             input_number_of_occupants,
                             input_number_of_drivers,
                             input_number_of_cars, 
                             input_income,
                             input_w_income,
                             input_population,
                             input_w_population,
                             input_year_of_birth,
                             input_w_year_of_birth,
                             input_job,
                             input_w_job,
                             input_distance,
                             input_w_distance,
                             quantity)
print(households)

In [ ]:
# Rank all households

# Choose the following main parameters (will be fulfilled)
input_number_of_occupants = 2
input_number_of_drivers = 2
input_number_of_cars = 2


# Choose the following five soft parameters and their weight.
# Fitting households will be sorted by weighted soft parameters.
# Sum of all weights = 1.0
# Find codes in Codeplan_SOC_Profile_Generation.md on github
input_income = 1
input_w_income = 0

input_population = 9
input_w_population = 0

input_year_of_birth = 1980
input_w_year_of_birth = 0

input_job = 1
input_w_job = 0

# [km / week]
input_distance = 100
input_w_distance = 1

# How many households should be returned?
# Put "all", if all fitting households should be returned
quantity = "all"

###

all_households = rank_households_all(meta_mop_data,
                             states_mop_data,
                             speed_mop_data,
                             no_of_ts, 
                             ts_length,
                             input_number_of_occupants,
                             input_number_of_drivers,
                             input_number_of_cars, 
                             input_income,
                             input_w_income,
                             input_population,
                             input_w_population,
                             input_year_of_birth,
                             input_w_year_of_birth,
                             input_job,
                             input_w_job,
                             input_distance,
                             input_w_distance,
                             quantity)
print(len(all_households))

### SOC Profile Generation

In [ ]:
### Create SOC profiles for each car from every household

'''
Choose the following parameters:
- household_list:           put "households" for whole list from rank_households
- start, end:               first and last timestep
- home_charging_power:      possible charging power at home [kW]
- work_charging_power:      -"- at work -> only one of both can be 0 [kW]
- charging_efficiency:      efficiency while charging
- discharging_efficiency:   efficiency while discharging (= consuming)
- min_charge:               SOC can not go lower (relative to battery capacity)
- max_charge:               SOC can not go higher (relative to battery capacity)
- path:                     path where csv-outputs should be saved
- bool_plot:                true, if plots should be created
- bool_create_csv:          true, if csv-files should be created
'''

household_list = households
start = 0
end = 1008
home_charging_power = 3.7
work_charging_power = 3.7
charging_efficiency = 0.95
discharging_efficiency = 0.95
min_SOC = 0.1
max_SOC = 0.9
bool_plot = True
bool_create_csv = False
path_csv = path_directory + '/outputs/'

###

create_soc_profiles(household_list,
                meta_mop_data,
                states_mop_data,
                speed_mop_data,
                start,
                end,
                no_of_ts,
                ts_length,
                home_charging_power,
                work_charging_power,
                charging_efficiency,
                discharging_efficiency,
                min_SOC,
                max_SOC,
                weather_csv,
                mob_car_info_csv,
                el_cars_csv,
                path_csv,
                bool_plot,
                bool_create_csv)

### Aggregated Load Profiles: PLOT 1 (24h)

In [ ]:
# PLOT 1: Aggregated Profile for one day (Weekday and Weekend)

household_list = all_households
start_weekday = 0
end_weekday = 1008
start_weekend = 0
end_weekend = 1008
home_charging_power = 3.7
work_charging_power = 3.7
charging_efficiency = 0.95
discharging_efficiency = 0.95
min_SOC = 0.1
max_SOC = 0.9
bool_winter = False
bool_plot = True

###
weekday = aggregated_profiles_day(household_list,
                meta_mop_data,
                states_mop_data,
                speed_mop_data,
                start_weekday,
                end_weekday,
                no_of_ts,
                ts_length,
                home_charging_power,
                work_charging_power,
                charging_efficiency,
                discharging_efficiency,
                min_SOC,
                max_SOC,
                weather_csv,
                mob_car_info_csv,
                el_cars_csv,
                bool_winter,
                bool_plot)

weekend = aggregated_profiles_day(household_list,
                meta_mop_data,
                states_mop_data,
                speed_mop_data,
                start_weekend,
                end_weekend,
                no_of_ts,
                ts_length,
                home_charging_power,
                work_charging_power,
                charging_efficiency,
                discharging_efficiency,
                min_SOC,
                max_SOC,
                weather_csv,
                mob_car_info_csv,
                el_cars_csv,
                bool_winter,
                bool_plot)


fig, ax = plt.subplots(figsize=(12,6))
figure_title = ("\nAggregierte Last")

ax.plot(weekday [432:576], color = "royalblue", label = "Werktag")
ax.plot(weekend [864:1008], color = "darkblue", label = "Wochenende")

ax.set_xlabel("Zeit")
ax.set_ylabel("Last normiert [kW]")
 
#ax.set_xticks(np.arange(min(ax.get_xticks()),max(ax.get_xticks()),36));
#labels = ['00:00 Uhr', '06:00 Uhr', '12:00 Uhr', '18:00 Uhr' , '00:00 Uhr']
#ax.set_xticklabels(labels, rotation = 0)

ax.set_xlim(0,144)
fig.legend(bbox_to_anchor = (1.2,0.85), loc="upper right")
ax.set_title(figure_title)
plt.tight_layout()
#plt.savefig("...", bbox_inches='tight')
plt.show()

### Aggregated Load Profiles: PLOT 2 (7 days)

In [ ]:
# PLOT 2: Aggregated Profile for one week

household_list = all_households
start = 0
end = 1008
home_charging_power = 3.7
work_charging_power = 3.7
charging_efficiency = 0.95
discharging_efficiency = 0.95
min_SOC = 0.1
max_SOC = 0.9
bool_winter = False
bool_plot = True

###
aggregated_profiles_week(household_list,
                meta_mop_data,
                states_mop_data,
                speed_mop_data,
                start,
                end,
                no_of_ts,
                ts_length,
                home_charging_power,
                work_charging_power,
                charging_efficiency,
                discharging_efficiency,
                min_SOC,
                max_SOC,
                weather_csv,
                mob_car_info_csv,
                el_cars_csv,
                bool_winter,
                bool_plot)

### Aggregated Profiles: PLOT 3 (Lastprofil Ladestrategien)

In [ ]:
# PLOT 3: Aggregated Profile for Max and Min Strategy

household_list = all_households
start = 0
end = 1008
home_charging_power = 3.7
work_charging_power = 3.7
charging_efficiency = 0.95
discharging_efficiency = 0.95
min_SOC = 0.1
max_SOC = 0.9
bool_winter = False
bool_plot = True

###

aggregated_profiles_strategies(household_list,
                meta_mop_data,
                states_mop_data,
                speed_mop_data,
                start,
                end,
                no_of_ts,
                ts_length,
                home_charging_power,
                work_charging_power,
                charging_efficiency,
                discharging_efficiency,
                min_SOC,
                max_SOC,
                weather_csv,
                mob_car_info_csv,
                el_cars_csv,
                bool_winter,
                bool_plot)

### Aggregated Profiles: PLOT 4 (Lastprofil Winter Sommer)

In [ ]:
household_list = all_households
start = 0
end = 1008
home_charging_power = 3.7
work_charging_power = 3.7
charging_efficiency = 0.95
discharging_efficiency = 0.95
min_SOC = 0.1
max_SOC = 0.9
bool_winter = False
bool_plot = True
bool_create_csv = False
path_csv = path_directory + '/outputs/'

###

summer = aggregated_profiles_day(household_list,
                meta_mop_data,
                states_mop_data,
                speed_mop_data,
                start,
                end,
                no_of_ts,
                ts_length,
                home_charging_power,
                work_charging_power,
                charging_efficiency,
                discharging_efficiency,
                min_SOC,
                max_SOC,
                weather_csv,
                mob_car_info_csv,
                el_cars_csv,
                bool_winter,
                bool_plot)

bool_winter = True
winter = aggregated_profiles_day(household_list,
                meta_mop_data,
                states_mop_data,
                speed_mop_data,
                start,
                end,
                no_of_ts,
                ts_length,
                home_charging_power,
                work_charging_power,
                charging_efficiency,
                discharging_efficiency,
                min_SOC,
                max_SOC,
                weather_csv,
                mob_car_info_csv,
                el_cars_csv,
                bool_winter,
                bool_plot)

fig, ax = plt.subplots(figsize=(12,6))
figure_title = ("\nAggregierte Last")
ax.plot(winter[144:288], color = "blue", label = "Winter")
ax.plot(summer[144:288], color = "red", label = "Sommer")

print("Summer:", sum(summer))
print("Winter:", sum(winter))

ax.set_xlim(0,144)
ax.set_xlabel("Zeit")
ax.set_ylabel("Last normiert [kW]")
plt.legend(bbox_to_anchor = (1.05, 1), loc = 'upper left')
ax.set_title(figure_title)
#ax.set_xticks(np.arange(min(ax.get_xticks()),max(ax.get_xticks()),36));
#labels = ['00:00 Uhr', '06:00 Uhr', '12:00 Uhr', '18:00 Uhr' , '00:00 Uhr']
#ax.set_xticklabels(labels, rotation = 0)

plt.tight_layout()
#plt.savefig("...", bbox_inches='tight')
plt.show()

### Aggregated Profiles: PLOT 5 (Lastverschiebepotenzial)

In [ ]:
# PLOT 5: LVP

### Create aggregated load profiles for all cars and all households

household_list = all_households
start_weekday = 0
end_weekday = 1008
start_weekend = 720
end_weekend = 888
home_charging_power = 3.7
work_charging_power = 3.7
charging_efficiency = 0.95
discharging_efficiency = 0.95
min_SOC = 0.1
max_SOC = 0.9
bool_winter = False
bool_plot = True

###
aggregated_profiles_lvp(household_list,
                meta_mop_data,
                states_mop_data,
                speed_mop_data,
                start_weekday,
                end_weekday,
                no_of_ts,
                ts_length,
                home_charging_power,
                work_charging_power,
                charging_efficiency,
                discharging_efficiency,
                min_SOC,
                max_SOC,
                weather_csv,
                mob_car_info_csv,
                el_cars_csv,
                bool_winter,
                bool_plot)

### Aggregated Profiles: PLOT 6 (Aggregated Profiles Home / Work)


In [ ]:
household_list = all_households
start_weekday = 0
end_weekday = 1008

home_charging_power_1 = 3.7
work_charging_power_1 = 0
home_charging_power_2 = 0
work_charging_power_2 = 3.7
home_charging_power_3 = 3.7
work_charging_power_3 = 3.7
charging_efficiency = 0.95
discharging_efficiency = 0.95
min_SOC = 0.1
max_SOC = 0.9
bool_winter = False
bool_plot = True

###
home_profile = aggregated_profiles_day(household_list,
                meta_mop_data,
                states_mop_data,
                speed_mop_data,
                start_weekday,
                end_weekday,
                no_of_ts,
                ts_length,
                home_charging_power_1,
                work_charging_power_1,
                charging_efficiency,
                discharging_efficiency,
                min_SOC,
                max_SOC,
                weather_csv,
                mob_car_info_csv,
                el_cars_csv,
                bool_winter,
                bool_plot)

work_profile = aggregated_profiles_day(household_list,
                meta_mop_data,
                states_mop_data,
                speed_mop_data,
                start_weekday,
                end_weekday,
                no_of_ts,
                ts_length,
                home_charging_power_2,
                work_charging_power_2,
                charging_efficiency,
                discharging_efficiency,
                min_SOC,
                max_SOC,
                weather_csv,
                mob_car_info_csv,
                el_cars_csv,
                bool_winter,
                bool_plot)

agg = aggregated_profiles_day(household_list,
                meta_mop_data,
                states_mop_data,
                speed_mop_data,
                start_weekday,
                end_weekday,
                no_of_ts,
                ts_length,
                home_charging_power_3,
                work_charging_power_3,
                charging_efficiency,
                discharging_efficiency,
                min_SOC,
                max_SOC,
                weather_csv,
                mob_car_info_csv,
                el_cars_csv,
                bool_winter,
                bool_plot)

fig, ax = plt.subplots(figsize=(12,6))
figure_title = ("\nAggregierte Last")

ax.plot(home_profile [432:576], color = "tomato", label = "Laden zu Hause")
ax.plot(work_profile [432:576], color = "maroon", label = "Laden auf Arbeit")
ax.plot(agg [432:576], color = "grey", label = "beide Ladestandorte", alpha =0.5)
ax.set_xlim(0,144)
ax.set_xlabel("Zeit")
ax.set_ylabel("Last normiert [kW]")
plt.legend(bbox_to_anchor = (1.05, 1), loc = 'upper left')
ax.set_title(figure_title)
#ax.set_xticks(np.arange(min(ax.get_xticks()),max(ax.get_xticks()),36));
#labels = ['00:00 Uhr', '06:00 Uhr', '12:00 Uhr', '18:00 Uhr' , '00:00 Uhr']
#ax.set_xticklabels(labels, rotation = 0)
plt.tight_layout()
#plt.savefig("...", bbox_inches='tight')
plt.show()